In [3]:
import os
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco

Technology: Megadam price

Source: https://doi.org/10.1016/j.enpol.2013.10.069

In [7]:
## Get current working directory
print("Current working directory:", os.getcwd())

## Find path to raw data (for the raw files)
raw_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'raw_data'))

## Find path to folder for saving cleaned csv
cleaned_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'cleaned_data'))

## Find path to folder for inflation
inflation_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'inflation'))

Current working directory: /Users/jennagreene/Documents/GitHub/HATCH_data/reading_files


In [5]:
## ADJUST BASED ON EACH TECHNOLOGY

# Find and read the CSV file from the raw_data folder
target_file = 'cost data/Ansar et al.xlsx'
target_file_path = os.path.join(raw_data_path, target_file)

target_file_path

'/Users/jennagreene/Documents/GitHub/HATCH_data/raw_data/cost data/Ansar et al.xlsx'

In [8]:
target_inflation_file = 'A001RG3A086NBEA.xls'
target_inflation_path = os.path.join(inflation_data_path, target_inflation_file)
target_inflation_path

'/Users/jennagreene/Documents/GitHub/HATCH_data/inflation/A001RG3A086NBEA.xls'

Adjust for inflation

In [10]:
## adjusting for inflation from 2012 USD to 2022 USD
nipa = pd.read_excel(target_inflation_path, header=10)

year_list = []
for x in nipa['observation_date']:
    x = int(str(x)[:4])
    year_list.append(x)
nipa['Year'] = year_list
nipa.set_index('Year', drop=True, inplace=True)
nipa.drop(columns='observation_date', inplace=True)
nipa = nipa.transpose()

infl_2012_2022 = float(nipa[2022]/nipa[2012])
infl_2012_2022

/var/folders/cy/xd4mmw610b3cr6tjgv0h8hlr0000gn/T/ipykernel_78368/1553214070.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  infl_2012_2022 = float(nipa[2022]/nipa[2012])


1.27197

Transpose and add characteristics to data

In [11]:
df = pd.read_excel(target_file_path, usecols=[0,1,13])
df = df[df['Source']=='Hydro']
num_list = []
for x in df['Cost/kW**']:
    num = float(x.replace(',','.'))
    num_list.append(num)
df['2012 USD/kW'] = num_list
df.drop(columns=['Source', 'Cost/kW**'], inplace=True)
df = df.groupby(['Date']).mean()
df = df.transpose()
# adjust for inflation
df.iloc[0] = df.iloc[0] * infl_2012_2022
df['Data Source'] = 'Ansar et al.'
df['Country Name'] = 'World'
df['Country Code'] = 'World'
df['Spatial Scale'] = 'Global'
df['Metric'] = 'Price'
df['Technology Name'] = 'Hydropower Mega-Dams'
df['Unit'] = '2022 USD/kW'
df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
df.set_index('ID', drop=True, inplace=True)
df

Date,1936,1942,1948,1950,1953,1956,1959,1963,1968,1970,...,2011,2012,2013,Data Source,Country Name,Country Code,Spatial Scale,Metric,Technology Name,Unit
ID,,,,,,,,,,,,,,,,,,,,,
Hydropower Mega-Dams_Price_World,1707.746922,2501.583399,186.725196,189.396333,196.519365,5331.716649,3900.877596,234.424071,2316.002976,3137.94999,...,1662.401192,3362.579892,13176.973215,Ansar et al.,World,World,Global,Price,Hydropower Mega-Dams,2022 USD/kW


Save data

In [12]:
## Change file name to save

megadams_price = df
output_file = 'megadams_price.csv'

output_file_path = os.path.join(cleaned_data_path, output_file)

megadams_price.to_csv(output_file_path)
print("Data saved to:", output_file_path)

Data saved to: /Users/jennagreene/Documents/GitHub/HATCH_data/cleaned_data/megadams_price.csv
